In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
torch.manual_seed(42)
import matplotlib.pyplot as plt
from PIL import Image
import skimage
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
class relu_mlp(nn.Module):
  def __init__(self, neurons, h_layers):
    super(relu_mlp, self).__init__()
    self.neurons = neurons
    self.h_layers = h_layers

    self.layers = nn.ModuleList()
    self.layers.append(nn.Linear(2,neurons))
    for i in range (h_layers-1):
      self.layers.append(nn.Linear(neurons,neurons))
    self.layers.append(nn.Linear(neurons, 3))
  
  def forward (self,x):
    for layer in self.layers[:-1]:
      x = F.relu(layer(x))
    x = self.layers[-1](x)
    return x

In [ ]:
class siren_mlp(nn.Module):
  def __init__(self, neurons, h_layers, omega = 30, first_omega = 30):
    super(siren_mlp, self).__init__()
    self.neurons = neurons
    self.h_layers = h_layers
    self.omega = omega
    self.first_omega = first_omega

    self.layers = nn.ModuleList()
    self.layers.append(nn.Linear(1, neurons))
    for i in range(h_layers):
      self.layers.append(nn.Linear(neurons, neurons))
    self.layers.append(nn.Linear(neurons,1))

    nn.init.uniform_(self.layers[0].weight,-1/1,1/1)
    self.layers[0].weight.data *= first_omega

    for layer in self.layers[1:]:
      nn.init.uniform_(layer.weight,-np.sqrt(6/neurons)/omega, np.sqrt(6/neurons)/omega)
      layer.weight.data *= omega
    
  
  def forward (self, x):
    for layer in self.layers[:-1]:
      x = torch.sin(layer(x))
    x = self.layers[-1](x)
    return x

In [ ]:
import scipy.io.wavfile as wavfile
import io
from IPython.display import Audio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
data = wavfile.read('gt_bach.wav')
Audio(data[1], rate = data[0])

In [ ]:
data[0]

44100

In [ ]:
data = data[1].astype(np.float32)

In [ ]:
data

array([0.        , 0.        , 0.        , ..., 0.04731218, 0.02968716,
       0.01679885], dtype=float32)

In [ ]:
np.min(data)

-0.868308

In [ ]:
len(data)

308207

In [ ]:
X = np.arange(1,len(data)+1)
y = data

In [ ]:
X

array([     1,      2,      3, ..., 308205, 308206, 308207])

In [ ]:
y = y

In [ ]:
np.min(y)

-0.868308

In [ ]:
device = 'cuda'
torch.cuda.is_available()

True

In [ ]:
siren_model = siren_mlp(256,3,30, 3000).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(siren_model.parameters(), lr=5*10**(-5))

In [ ]:
siren_psnr = []

In [ ]:
from tqdm import tqdm

In [ ]:
X = X.reshape(-1,1)

In [ ]:
num_epochs = 5000
for e in tqdm(range(num_epochs)):
  y_pred_train = siren_model(torch.from_numpy(X.astype('float32')).to(device)).squeeze()
  loss = criterion(y_pred_train, torch.from_numpy(y.astype('float32')).to(device)) # loss is a tensor which stores the current value of train loss
  siren_psnr.append(20 * np.log10(1.0 / np.sqrt(loss.item())))

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if e % 100 == 0:
    print(f"Epoch [{e}/{num_epochs}], Loss: {loss.item():.4f}") # loss.item() gives the numerical value stored in loss
  if e == num_epochs-1:
    print(f"Epoch [{e}/{num_epochs}], Loss: {loss.item():.4f}") # loss.item() gives the numerical value stored in loss
     

  0%|          | 1/5000 [00:00<16:19,  5.10it/s]

Epoch [0/5000], Loss: 0.0107


  2%|▏         | 101/5000 [00:19<19:15,  4.24it/s]

Epoch [100/5000], Loss: 0.0107


  4%|▍         | 201/5000 [00:38<18:12,  4.39it/s]

...

Epoch [4800/5000], Loss: 0.0053


 98%|█████████▊| 4901/5000 [15:08<00:21,  4.53it/s]

Epoch [4900/5000], Loss: 0.0052


100%|██████████| 5000/5000 [15:26<00:00,  5.40it/s]

Epoch [4999/5000], Loss: 0.0052


In [ ]:
audio_siren = siren_model(torch.from_numpy(X.astype('float32')).to(device))

In [ ]:
audio_siren = audio_siren.cpu().detach().numpy().squeeze()

In [ ]:
audio_siren

array([ 0.0397495 , -0.02673191, -0.06978832, ...,  0.08356562,
        0.02265902, -0.06054626], dtype=float32)

In [ ]:
Audio(audio_siren, rate = 44100)

In [ ]:
y

array([0.        , 0.        , 0.        , ..., 0.04731218, 0.02968716,
       0.01679885], dtype=float32)

In [ ]:
y_pred_train = y_pred_train.cpu()

In [ ]:
y_pred_train = y_pred_train.detach().numpy()

In [ ]:
y_pred_train

array([ 0.04028275, -0.02608917, -0.0689289 , ...,  0.12037916,
        0.02758162, -0.10478705], dtype=float32)

In [ ]:
Audio(y_pred_train, rate = 44100)

In [ ]:
from scipy.io.wavfile import write
write('siren_audio.wav', 44100, y_pred_train)

In [ ]:
X_extrapolated = X + 308207

In [ ]:
audio_siren_extrapolated = siren_model(torch.from_numpy(X_extrapolated.astype('float32')).to(device))

In [ ]:
audio_siren_extrapolated = audio_siren_extrapolated.cpu().detach().numpy().squeeze()

In [ ]:
audio_siren_extrapolated

array([-0.07883598, -0.04245985, -0.07184346, ...,  0.1567958 ,
        0.23646842,  0.01340024], dtype=float32)

In [ ]:
Audio(audio_siren_extrapolated, rate = 44100)

In [ ]:
write('siren_audio_extrapolated', 44100, audio_siren_extrapolated)

In [ ]:
torch.save(siren_model.state_dict(), 'MyDrive')